# Lecture 1: overview of tensorflow

**Useful links:** 
1. [Lecture note](http://web.stanford.edu/class/cs20si/lectures/notes_01.pdf);
2. [Leture slides](http://web.stanford.edu/class/cs20si/lectures/slides_01.pdf);
3. [A blog: What is a TensorFlow Session?](http://danijar.com/what-is-a-tensorflow-session/);
4. [Quora: How does graph creation work in TensorFlow](https://www.quora.com/How-does-graph-creation-work-in-TensorFlow);

In [1]:
import tensorflow as tf
import numpy as np

# Part 1

## Graphs and Session
TensorFlow separates definition of computations from their execution
1. definition your computations by assembling a graph
2. execute your computations by running a (sub)graph in a session

### Tensor
Tensor is an n-dimensional matrix
1. 0-d tensor: scalar(number)
2. 1-d tensor: vector
3.  2-d tensor: matrix
4. and so no (an rgb image can be 3-d tensor and an rgbd image can be 4-d tensor)

In [2]:
a = tf.add(3, 5) # tf.int32 implicitly
print(a)
print(a.op.name)
# Not 8: We just defince a graph. To get the value of a, we nend to create a session and
# run the graph in the sesssion

Tensor("Add:0", shape=(), dtype=int32)
Add


In [3]:
b = tf.add(3.0, 5.0) # tf.float32 implicitly
print(b)

Tensor("Add_1:0", shape=(), dtype=float32)


In [4]:
sess = tf.Session() # Without arguments the session constructor launches the default graph
print(sess.run(a)) # run the graph, now we get the value of a
#print(sess.run(b))
sess.close()

8


<bound method Session.close of <tensorflow.python.client.session.Session object at 0x7fbc54384050>>

In [5]:
# use 'with as' statement to make your code neat
with tf.Session() as sess:
    print(sess.run(a))

8


### running a graph or a subgraph

In [ ]:
# defining a graph
x = 2
y = 3
op1 = tf.add(x, y)
op2 = tf.mul(x, y)
op3 = tf.pow(op2, op1)
# tf.mul, tf.sub and tf.neg are replaced by tf.multiply, tf.substract and tf.negative
# since my tensorflow version is 1.0.0, it raise a error
# you might replace tf.mul by tf.multiply if you are using 1.0.0
# to get your version by:
# python -c 'import tensorflow as tf; print(tf.__version__)'  # for Python 2
# python3 -c 'import tensorflow as tf; print(tf.__version__)'  # for Python 3

In [6]:
# replace tf.mul by tf.multiply if you are using 1.0.0
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(2) # tf.constant() will build a node
    y = tf.constant(3)
    op1 = tf.add(x, y)
    op2 = tf.multiply(x, y)
    op3 = tf.pow(op2, op1)
print(graph.get_operations()) # Return the list of operations in the graph.
print(graph.version) #Returns a version number that increases as ops are added to the graph.

[<tf.Operation 'Const' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'Add' type=Add>, <tf.Operation 'Mul' type=Mul>, <tf.Operation 'Pow' type=Pow>]
5


In [7]:
# run the graph
with tf.Session(graph = graph) as sess:
    op3 = sess.run(op3)
    print(op3) # 6^5 = 7776

7776


#### running a subgraph example

In [8]:
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(2)
    y = tf.constant(3)
    op1 = tf.add(x, y)
    op2 = tf.multiply(x, y)
    useless = tf.multiply(x, op1) # this multipy operation wolud not be run in case 1
    op3 = tf.pow(op2, op1)
print(graph.get_operations())
print(graph.version)

[<tf.Operation 'Const' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'Add' type=Add>, <tf.Operation 'Mul' type=Mul>, <tf.Operation 'Mul_1' type=Mul>, <tf.Operation 'Pow' type=Pow>]
6


In [9]:
# case 1
with tf.Session(graph = graph) as sess:
    op3 = sess.run(op3)
    print(op3)
    #print(useless)

7776


In [10]:
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(2)
    y = tf.constant(3)
    op1 = tf.add(x, y)
    op2 = tf.multiply(x, y)
    useless = tf.multiply(x, op1) # run this operation
    op3 = tf.pow(op2, op1)
print(graph.get_operations())
print(graph.version)

[<tf.Operation 'Const' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'Add' type=Add>, <tf.Operation 'Mul' type=Mul>, <tf.Operation 'Mul_1' type=Mul>, <tf.Operation 'Pow' type=Pow>]
6


In [11]:
# case 2
with tf.Session(graph = graph) as sess:
    op3, not_useless = sess.run([op3, useless])
    print(op3)
    print(not_useless)

7776
10


## Distributed comptation

In [12]:
with tf.device('/gpu:2'):
    graph = tf.Graph()
    with graph.as_default():
        a = tf.constant(np.array([[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]]), name='a')
        b = tf.constant(np.array([[1.0], 
                                  [2.0], 
                                  [3.0], 
                                  [4.0], 
                                  [5.0], 
                                  [6.0]]), name='b')
        c = tf.matmul(a, b)
    print(graph.get_operations())
    print(graph.version)

[<tf.Operation 'a' type=Const>, <tf.Operation 'b' type=Const>, <tf.Operation 'MatMul' type=MatMul>]
3


In [13]:
with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as sess:
    c = sess.run(c)
    print(c)

[[ 91.]]


In [22]:
with tf.device('/gpu:2'):
    graph = tf.Graph()
    with graph.as_default():
        a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='a', shape=[2, 3])
        b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='b', shape=[3, 2])
        c = tf.matmul(a, b)
    print(graph.get_operations())
    print(graph.version)

[<tf.Operation 'a' type=Const>, <tf.Operation 'b' type=Const>, <tf.Operation 'MatMul' type=MatMul>]
3


In [23]:
with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as sess:
    c = sess.run(c)
    print(c)

[[ 22.  28.]
 [ 49.  64.]]


In [24]:
c = []
for d in ['/gpu:2', '/gpu:3']:
  with tf.device(d):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3])
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
    c.append(tf.matmul(a, b))
with tf.device('/cpu:0'):
  sum = tf.add_n(c)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print sess.run(sum)

[[  44.   56.]
 [  98.  128.]]


## More about graph

In [14]:
g = tf.Graph()
with g.as_default():
    a = tf.constant(3)
    b = tf.constant(5)
    x = tf.add(a, b)
print(g.get_operations())
print(g.version)

with tf.Session(graph = g) as sess:
    x = sess.run(x)
    print(x)

[<tf.Operation 'Const' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'Add' type=Add>]
3
8


In [15]:
# Do not mix default graph and user created graphs
g = tf.Graph()
a = tf.constant(3) # this op will be placed into default graph
print(a)
with g.as_default():
    b = tf.constant(5)
print(g.get_operations())
print(g.version)
# we only get one operation in g

Tensor("Const:0", shape=(), dtype=int32)
[<tf.Operation 'Const' type=Const>]
1


In [16]:
g1 = tf.Graph()
print(g1.get_operations())
print(g1.version)
g2 = tf.Graph()
print(g2.get_operations())
print(g2.version)

[]
0
[]
0


In [17]:
with g1.as_default():
    a = tf.constant(3)
with g2.as_default():
    b = tf.constant(5)
print(g1.get_operations())
print(g1.version)
print(g2.get_operations())
print(g2.version)

[<tf.Operation 'Const' type=Const>]
1
[<tf.Operation 'Const' type=Const>]
1


# Part 2

## Defining a computation graph

In [18]:
graph = tf.Graph()
with graph.as_default():
    variable = tf.Variable(42, name='foo')
    initialize = tf.global_variables_initializer()
    assign = variable.assign(13)
print(graph.get_operations())
print(graph.version)

[<tf.Operation 'foo/initial_value' type=Const>, <tf.Operation 'foo' type=VariableV2>, <tf.Operation 'foo/Assign' type=Assign>, <tf.Operation 'foo/read' type=Identity>, <tf.Operation 'init' type=NoOp>, <tf.Operation 'Assign/value' type=Const>, <tf.Operation 'Assign' type=Assign>]
7


### print them out to get some intution

In [19]:
print(graph)
print 
print(variable)
print
print(initialize)
print
print(assign)


Tensor("foo/read:0", shape=(), dtype=int32)

name: "init"
op: "NoOp"
input: "^foo/Assign"


Tensor("Assign:0", shape=(), dtype=int32_ref)


## Running defined graph in a Session

In [20]:
with tf.Session(graph = graph) as sess:
    sess.run(initialize)
    sess.run(assign)
    print(sess.run(variable))

13


In [ ]:
with tf.Session(graph = graph) as sess:
    print(sess.run(variable))
    
# FailedPreconditionError: Attempting to use uninitialized value foo

In [21]:
with tf.Session(graph = graph) as sess:
    sess.run(initialize)
    print(sess.run(variable))

42
